In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set(style='ticks', palette='RdBu')
#sns.set(style='ticks', palette='Set2')
import datetime
import pytz
import businesstimedelta
import holidays as pyholidays
import time as t
from workalendar.africa import Kenya
from datetime import date
%matplotlib inline

In [2]:
# 3rd floor all user updates
status = pd.read_excel(r"C:\Users\user\Documents\Alex\STB Report\note book\Mitigation\3rd floor all user status.xlsx")

# Branch All User Updates
branch = pd.read_excel(r"C:\Users\user\Documents\Alex\STB Report\note book\Mitigation\Branch all user updates.xlsx")

In [3]:
# Merging branch and Hq all updates
status = pd.concat([status,branch])

# Merging and changing time format
status['Date'] = pd.to_datetime(status['Date'], dayfirst=True)
status['Datetime'] = status['Date'].astype(str)+' '+status['Time'].astype(str)
status = status.dropna(subset='Time')
status['Datetime'] = pd.to_datetime(status['Datetime'], format = '%Y/%m/%d %H:%M:%S')
status.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime
0,approvals1,2023-02-01,14:15:00,Draft Payments Posted,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00
1,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00


### Defining a working day

In [4]:
status.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime
0,approvals1,2023-02-01,14:15:00,Draft Payments Posted,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00
1,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00
2,approvals1,2023-02-01,14:17:00,Draft Payments Posted,942637,234300684,Order with New Frame from Branch and Lens from HQ,2023-02-01 14:17:00


In [5]:
status.to_clipboard()

# MAINSTORE-allmain

## MAINSTORE(order printed to lens store)-mainstore_orderprinted_lenstore

In [6]:
sales_order = status[(status['Status']== 'Sales Order Created') | (status['Status']== 'Confirmed by Approvals Team')]
sales_order = sales_order.drop_duplicates(subset=['DocEntry','Status'], keep='first')
sales_order['Status'].unique()


array(['Confirmed by Approvals Team', 'Sales Order Created'], dtype=object)

In [7]:
#Filtering to Frame sent to lenstore
senttolens = status[status['Status']== 'Frame Sent to Lens Store']
senttolens = senttolens.drop_duplicates(subset='DocEntry', keep='first')

In [8]:
senttolens1 = senttolens[senttolens['Created User']== 'main1']
senttolens2 = senttolens[senttolens['Created User']== 'main2']
senttolens3 = senttolens[senttolens['Created User']== 'main3']

senttolens=pd.concat([senttolens1,senttolens2,senttolens3])

senttolens['Status'].unique()

array(['Frame Sent to Lens Store'], dtype=object)

In [9]:
senttolens.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [10]:
orderprinted_lensstore= pd.merge(sales_order,senttolens[['DocEntry','Datetimeout']], on='DocEntry', how='right')
orderprinted_lensstore.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,ohoc2,2023-02-04,12:40:00,Confirmed by Approvals Team,947501,23204488.0,Frame from HQ and Lens from HQ,2023-02-04 12:40:00,2023-02-04 12:52:00
1,approvals1,2023-02-04,12:45:00,Confirmed by Approvals Team,947549,235100395.0,Frame from HQ and Lens from HQ,2023-02-04 12:45:00,2023-02-04 12:52:00
2,yorc2,2023-02-04,12:44:00,Confirmed by Approvals Team,947680,23103173.0,Frame from HQ and Lens from HQ,2023-02-04 12:44:00,2023-02-04 12:53:00


In [11]:
orderprinted_lensstore=orderprinted_lensstore.dropna(subset='Datetime')

In [12]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=orderprinted_lensstore.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [13]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=orderprinted_lensstore.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [14]:
orderprinted_lensstore["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

orderprinted_lensstore =orderprinted_lensstore.replace("", np.nan)


In [15]:
orderprinted_lensstore['delayed dept']=orderprinted_lensstore['delay'].apply(lambda x: 'Delayed at Main store' if x>10 else 'Did not delay at Main store' )
orderprinted_lensstore.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,ohoc2,2023-02-04,12:40:00,Confirmed by Approvals Team,947501,23204488.0,Frame from HQ and Lens from HQ,2023-02-04 12:40:00,2023-02-04 12:52:00,12.0,Delayed at Main store
1,approvals1,2023-02-04,12:45:00,Confirmed by Approvals Team,947549,235100395.0,Frame from HQ and Lens from HQ,2023-02-04 12:45:00,2023-02-04 12:52:00,7.0,Did not delay at Main store


In [16]:
mainstore_orderprinted_lenstore=orderprinted_lensstore

In [17]:
mainstore_orderprinted_lenstore.to_clipboard()

## MAINSTORE(order printed to control room)-mainstore_orderprinted_control

In [18]:
sales_order = status[(status['Status']== 'Sales Order Created') | (status['Status']== 'Confirmed by Approvals Team')]
sales_order = sales_order.drop_duplicates(subset=['DocEntry','Status'], keep='first')
sales_order['Status'].unique()

array(['Confirmed by Approvals Team', 'Sales Order Created'], dtype=object)

In [19]:
#Filtering to Frame sent to lenstore
senttocontrol = status[status['Status']== 'Sent to Control Room']
senttocontrol = senttocontrol.drop_duplicates(subset='DocEntry', keep='first')

In [20]:
senttocontrol1 = senttocontrol[senttocontrol['Created User']== 'main1']
senttocontrol2 = senttocontrol[senttocontrol['Created User']== 'main2']
senttocontrol3 = senttocontrol[senttocontrol['Created User']== 'main3']

senttocontrol=pd.concat([senttocontrol1,senttocontrol2,senttocontrol3])

senttocontrol['Status'].unique()

array(['Sent to Control Room'], dtype=object)

In [21]:
senttocontrol.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [22]:
orderprinted_control= pd.merge(sales_order,senttocontrol[['DocEntry','Datetimeout']], on='DocEntry', how='left')
orderprinted_control.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00,2023-02-01 14:23:00
1,approvals1,2023-02-01,14:17:00,Confirmed by Approvals Team,942637,234300684,Order with New Frame from Branch and Lens from HQ,2023-02-01 14:17:00,NaT
2,approvals1,2023-02-01,14:20:00,Confirmed by Approvals Team,943009,235100349,Frame from HQ and Lens from HQ,2023-02-01 14:20:00,NaT


In [23]:
orderprinted_control=orderprinted_control.dropna(subset='Datetimeout')

In [24]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=orderprinted_control.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [25]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=orderprinted_control.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [26]:
orderprinted_control["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

orderprinted_control =orderprinted_control.replace("", np.nan)


In [27]:
orderprinted_control['delayed dept']=orderprinted_control['delay'].apply(lambda x: 'Delayed at Main store' if x>10 else 'Did not delay at Main store' )
orderprinted_control.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00,2023-02-01 14:23:00,8.0,Did not delay at Main store
215,approvals1,2023-02-10,15:19:00,Confirmed by Approvals Team,954755,232100340,Contact lens from HQ,2023-02-10 15:19:00,2023-02-10 15:34:00,15.0,Delayed at Main store


In [28]:
mainstore_orderprinted_control=orderprinted_control

In [29]:
mainstore_orderprinted_control.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00,2023-02-01 14:23:00,8.0,Did not delay at Main store
215,approvals1,2023-02-10,15:19:00,Confirmed by Approvals Team,954755,232100340,Contact lens from HQ,2023-02-10 15:19:00,2023-02-10 15:34:00,15.0,Delayed at Main store


## MAINSTORE(rejected frame sent to frame store to reissued frame for order)-mainstoredamage_reissued

In [30]:
mainrejected = status[status['Status']== 'Rejected Frame sent to Frame Store']
mainrejected = mainrejected.drop_duplicates(subset=['DocEntry','Status'], keep='first')
mainrejected['Status'].unique()

array(['Rejected Frame sent to Frame Store'], dtype=object)

In [31]:
reissuedframemain = status[status['Status']== 'ReIssued Frame for Order']
reissuedframemain['Status'].unique()

array(['ReIssued Frame for Order'], dtype=object)

In [32]:
reissuedframemain1 = reissuedframemain[reissuedframemain['Created User']== 'main1']
reissuedframemain2 = reissuedframemain[reissuedframemain['Created User']== 'main2']
reissuedframemain3 = reissuedframemain[reissuedframemain['Created User']== 'main3']

reissuedframemain=pd.concat([reissuedframemain1,reissuedframemain2,reissuedframemain3])

reissuedframemain['Status'].unique()

array(['ReIssued Frame for Order'], dtype=object)

In [33]:
reissuedframemain.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [34]:
reissueframemain= pd.merge(mainrejected,reissuedframemain[['DocNum','Datetimeout']], on='DocNum', how='left')
reissueframemain.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,control1,2023-02-01,10:56:00,Rejected Frame sent to Frame Store,943012,231300712,Surfacing Lens with HQ Frame,2023-02-01 10:56:00,2023-02-01 11:04:00
1,control1,2023-02-01,10:56:00,Rejected Frame sent to Frame Store,943012,231300712,Surfacing Lens with HQ Frame,2023-02-01 10:56:00,2023-02-01 11:04:00
2,control1,2023-02-02,11:47:00,Rejected Frame sent to Frame Store,944895,23204187,Surfacing Lens with HQ Frame,2023-02-02 11:47:00,2023-02-02 11:50:00


In [35]:
reissueframemain=reissueframemain.dropna(subset='Datetimeout')

In [36]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=reissueframemain.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [37]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=reissueframemain.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [38]:

reissueframemain["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

reissueframemain =reissueframemain.replace("", np.nan)


In [39]:
reissueframemain['delayed dept']=reissueframemain['delay'].apply(lambda x: 'Main store delayed to reissue Frame' if x>10 else 'Did not delay at Main store' )
reissueframemain.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,control1,2023-02-01,10:56:00,Rejected Frame sent to Frame Store,943012,231300712,Surfacing Lens with HQ Frame,2023-02-01 10:56:00,2023-02-01 11:04:00,8.0,Did not delay at Main store
1,control1,2023-02-01,10:56:00,Rejected Frame sent to Frame Store,943012,231300712,Surfacing Lens with HQ Frame,2023-02-01 10:56:00,2023-02-01 11:04:00,8.0,Did not delay at Main store


In [40]:
mainstoredamage_reissued=reissueframemain

## MAINSTORE(out of stock)- main_outofstock

In [41]:
sales_order = status[(status['Status']== 'Sales Order Created') | (status['Status']== 'Confirmed by Approvals Team')]
sales_order['Status'].unique()

orderprinted1 = sales_order
orderprinted2 = status[status['Status']== 'Rejected Frame sent to Frame Store']
orderprinted=pd.concat([orderprinted1,orderprinted2])
orderprinted = orderprinted.drop_duplicates(subset=['DocEntry','Status'], keep='first')
orderprinted['Status'].unique()

array(['Confirmed by Approvals Team', 'Sales Order Created',
       'Rejected Frame sent to Frame Store'], dtype=object)

In [42]:
outofstock = status[status['Status']== 'HQ Item Out of Stock Getting It']
outofstock['Status'].unique()

array(['HQ Item Out of Stock Getting It'], dtype=object)

In [43]:
outofstock1 = outofstock[outofstock['Created User']== 'main1']
outofstock2 = outofstock[outofstock['Created User']== 'main2']
outofstock3 = outofstock[outofstock['Created User']== 'main3']

outofstock=pd.concat([outofstock1,outofstock2,outofstock3])
outofstock = outofstock
outofstock['Status'].unique()

array(['HQ Item Out of Stock Getting It'], dtype=object)

In [44]:
outofstock.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [45]:
outofstock= pd.merge(orderprinted,outofstock[['DocNum','Datetimeout']], on='DocNum', how='left')
outofstock.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00,NaT
1,approvals1,2023-02-01,14:17:00,Confirmed by Approvals Team,942637,234300684,Order with New Frame from Branch and Lens from HQ,2023-02-01 14:17:00,NaT
2,approvals1,2023-02-01,14:20:00,Confirmed by Approvals Team,943009,235100349,Frame from HQ and Lens from HQ,2023-02-01 14:20:00,NaT


In [46]:
outofstock = outofstock.dropna(subset='Datetimeout')

In [47]:
outofstock['delayed dept']='HQ Item Out of Stock'
main_outofstock=outofstock
main_outofstock.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delayed dept
418,approvals1,2023-02-18,16:01:00,Confirmed by Approvals Team,963852,236400497,Frame from HQ and Lens from HQ,2023-02-18 16:01:00,2023-02-18 16:06:00,HQ Item Out of Stock
419,approvals1,2023-02-18,16:01:00,Confirmed by Approvals Team,963852,236400497,Frame from HQ and Lens from HQ,2023-02-18 16:01:00,2023-02-18 16:06:00,HQ Item Out of Stock


## All Mainstore

In [48]:
allmain=pd.concat([mainstore_orderprinted_lenstore,mainstore_orderprinted_control,mainstoredamage_reissued,main_outofstock])
allmain['delayed dept'].unique()

array(['Delayed at Main store', 'Did not delay at Main store',
       'Main store delayed to reissue Frame', 'HQ Item Out of Stock'],
      dtype=object)

# DesignerStore-alldesigner

## DesignerStore(order printed to lens store)-DesignerStore_orderprinted_lenstore

In [49]:
sales_order = status[(status['Status']== 'Sales Order Created') | (status['Status']== 'Confirmed by Approvals Team')]
sales_order = sales_order.drop_duplicates(subset=['DocEntry','Status'], keep='first')
sales_order['Status'].unique()


array(['Confirmed by Approvals Team', 'Sales Order Created'], dtype=object)

In [50]:
#Filtering to Frame sent to lenstore
senttolens = status[status['Status']== 'Frame Sent to Lens Store']
senttolens = senttolens.drop_duplicates(subset='DocEntry', keep='first')

In [51]:
senttolens1 = senttolens[senttolens['Created User']== 'designer1']
senttolens2 = senttolens[senttolens['Created User']== 'designer2']
senttolens3 = senttolens[senttolens['Created User']== 'designer3']

senttolens=pd.concat([senttolens1,senttolens2,senttolens3])

senttolens['Status'].unique()

array(['Frame Sent to Lens Store'], dtype=object)

In [52]:
senttolens.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [53]:
orderprinted_lensstore= pd.merge(sales_order,senttolens[['DocEntry','Datetimeout']], on='DocEntry', how='right')
orderprinted_lensstore.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,NaN,NaT,NaN,NaN,941526,NaN,NaN,NaT,2023-02-01 08:57:00
1,ohoc2,2023-02-01,09:19:00,Confirmed by Approvals Team,939090,23203486.0,Frame from HQ and Lens from HQ,2023-02-01 09:19:00,2023-02-01 09:29:00
2,NaN,NaT,NaN,NaN,943533,NaN,NaN,NaT,2023-02-01 10:51:00


In [54]:
orderprinted_lensstore=orderprinted_lensstore.dropna(subset='Datetime')

In [55]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=orderprinted_lensstore.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [56]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=orderprinted_lensstore.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [57]:
orderprinted_lensstore["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

orderprinted_lensstore =orderprinted_lensstore.replace("", np.nan)


In [58]:
orderprinted_lensstore['delayed dept']=orderprinted_lensstore['delay'].apply(lambda x: 'Delayed at Designer' if x>10 else 'Did not delay at Designer' )
orderprinted_lensstore.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
1,ohoc2,2023-02-01,09:19:00,Confirmed by Approvals Team,939090,23203486.0,Frame from HQ and Lens from HQ,2023-02-01 09:19:00,2023-02-01 09:29:00,10.0,Did not delay at Designer
3,approvals2,2023-02-01,10:47:00,Confirmed by Approvals Team,940435,236100403.0,Frame from HQ and Lens from HQ,2023-02-01 10:47:00,2023-02-01 10:51:00,4.0,Did not delay at Designer


In [59]:
DesignerStore_orderprinted_lenstore=orderprinted_lensstore

## Designer(order printed to control room)-designer_orderprinted_control

In [60]:
sales_order = status[(status['Status']== 'Sales Order Created') | (status['Status']== 'Confirmed by Approvals Team')]
sales_order = sales_order.drop_duplicates(subset=['DocEntry','Status'], keep='first')
sales_order['Status'].unique()

array(['Confirmed by Approvals Team', 'Sales Order Created'], dtype=object)

In [61]:
#Filtering to Frame sent to lenstore
senttocontrol = status[status['Status']== 'Sent to Control Room']
senttocontrol = senttocontrol.drop_duplicates(subset='DocEntry', keep='first')

In [62]:
senttocontrol1 = senttocontrol[senttocontrol['Created User']== 'designer1']
senttocontrol2 = senttocontrol[senttocontrol['Created User']== 'designer2']
senttocontrol3 = senttocontrol[senttocontrol['Created User']== 'designer3']

senttocontrol=pd.concat([senttocontrol1,senttocontrol2,senttocontrol3])

senttocontrol['Status'].unique()

array(['Sent to Control Room'], dtype=object)

In [63]:
senttocontrol.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [64]:
orderprinted_control= pd.merge(sales_order,senttocontrol[['DocEntry','Datetimeout']], on='DocEntry', how='left')
orderprinted_control.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00,NaT
1,approvals1,2023-02-01,14:17:00,Confirmed by Approvals Team,942637,234300684,Order with New Frame from Branch and Lens from HQ,2023-02-01 14:17:00,NaT
2,approvals1,2023-02-01,14:20:00,Confirmed by Approvals Team,943009,235100349,Frame from HQ and Lens from HQ,2023-02-01 14:20:00,NaT


In [65]:
orderprinted_control=orderprinted_control.dropna(subset='Datetimeout')

In [66]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=orderprinted_control.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [67]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=orderprinted_control.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [68]:
orderprinted_control["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

orderprinted_control =orderprinted_control.replace("", np.nan)

In [69]:
orderprinted_control['delayed dept']=orderprinted_control['delay'].apply(lambda x: 'Delayed at Designer' if x>10 else 'Did not delay at Designer' )
orderprinted_control.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
1128,approvals2,2023-02-01,17:53:00,Confirmed by Approvals Team,940978,235400524,Contact lens from HQ,2023-02-01 17:53:00,2023-02-01 17:56:00,3.0,Did not delay at Designer
8914,csman,2023-02-18,10:40:00,Sales Order Created,963485,23206447,Contact lens from HQ,2023-02-18 10:40:00,2023-02-18 11:10:00,30.0,Delayed at Designer


In [70]:
designer_orderprinted_control=orderprinted_control

## DESIGNER(rejected frame sent to frame store to reissued frame for order)-designerdamage_reissued

In [71]:
designerrejected = status[status['Status']== 'Rejected Frame sent to Frame Store']
designerrejected = designerrejected.drop_duplicates(subset=['DocEntry','Status'], keep='first')
designerrejected['Status'].unique()

array(['Rejected Frame sent to Frame Store'], dtype=object)

In [72]:
reissuedframedes = status[status['Status']== 'ReIssued Frame for Order']
reissuedframedes['Status'].unique()

array(['ReIssued Frame for Order'], dtype=object)

In [73]:
reissuedframedes1 = reissuedframedes[reissuedframedes['Created User']== 'designer1']
reissuedframedes2 = reissuedframedes[reissuedframedes['Created User']== 'designer2']
reissuedframedes3 = reissuedframedes[reissuedframedes['Created User']== 'designer3']

reissuedframedes=pd.concat([reissuedframedes1,reissuedframedes2,reissuedframedes3])

reissuedframedes['Status'].unique()

array(['ReIssued Frame for Order'], dtype=object)

In [74]:
reissuedframedes.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [75]:
reissuedframedes= pd.merge(designerrejected,reissuedframedes[['DocNum','Datetimeout']], on='DocNum', how='left')
reissuedframedes.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,control1,2023-02-01,10:56:00,Rejected Frame sent to Frame Store,943012,231300712,Surfacing Lens with HQ Frame,2023-02-01 10:56:00,NaT
1,control1,2023-02-02,11:47:00,Rejected Frame sent to Frame Store,944895,23204187,Surfacing Lens with HQ Frame,2023-02-02 11:47:00,NaT
2,control1,2023-02-04,15:13:00,Rejected Frame sent to Frame Store,947885,232100281,Frame from HQ and Lens from HQ,2023-02-04 15:13:00,NaT


In [76]:
reissuedframedes=reissuedframedes.dropna(subset='Datetimeout')

In [77]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=reissuedframedes.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [78]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=reissuedframedes.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [79]:
reissuedframedes["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

reissuedframedes =reissuedframedes.replace("", np.nan)

In [80]:
reissuedframedes['delayed dept']=reissuedframedes['delay'].apply(lambda x: 'Designer delayed to reissue Frame' if x>10 else 'Did not delay at Designer' )
reissuedframedes.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
5,control1,2023-02-08,12:55:00,Rejected Frame sent to Frame Store,941065,234900155,Overseas Lens with Branch Frame,2023-02-08 12:55:00,2023-02-08 13:01:00,6.0,Did not delay at Designer
6,control1,2023-02-08,12:55:00,Rejected Frame sent to Frame Store,941065,234900155,Overseas Lens with Branch Frame,2023-02-08 12:55:00,2023-02-08 13:01:00,6.0,Did not delay at Designer


In [81]:
designerdamage_reissued=reissuedframedes

## DESIGNER(out of stock)- des_outofstock

In [82]:
sales_order = status[(status['Status']== 'Sales Order Created') | (status['Status']== 'Confirmed by Approvals Team')]
sales_order['Status'].unique()

orderprinted1 = sales_order
orderprinted2 = status[status['Status']== 'Rejected Frame sent to Frame Store']
orderprinted=pd.concat([orderprinted1,orderprinted2])
orderprinted = orderprinted.drop_duplicates(subset=['DocEntry','Status'], keep='first')
orderprinted['Status'].unique()

array(['Confirmed by Approvals Team', 'Sales Order Created',
       'Rejected Frame sent to Frame Store'], dtype=object)

In [83]:
outofstock = status[status['Status']== 'HQ Item Out of Stock Getting It']
outofstock['Status'].unique()

array(['HQ Item Out of Stock Getting It'], dtype=object)

In [84]:
outofstock1 = outofstock[outofstock['Created User']== 'designer1']
outofstock2 = outofstock[outofstock['Created User']== 'designer2']
outofstock3 = outofstock[outofstock['Created User']== 'designer3']

outofstock=pd.concat([outofstock1,outofstock2,outofstock3])
outofstock = outofstock
outofstock['Status'].unique()

array(['HQ Item Out of Stock Getting It'], dtype=object)

In [85]:
outofstock.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [86]:
outofstock= pd.merge(orderprinted,outofstock[['DocNum','Datetimeout']], on='DocNum', how='left')
outofstock.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,approvals1,2023-02-01,14:15:00,Confirmed by Approvals Team,939205,236000215,Contact lens from HQ,2023-02-01 14:15:00,NaT
1,approvals1,2023-02-01,14:17:00,Confirmed by Approvals Team,942637,234300684,Order with New Frame from Branch and Lens from HQ,2023-02-01 14:17:00,NaT
2,approvals1,2023-02-01,14:20:00,Confirmed by Approvals Team,943009,235100349,Frame from HQ and Lens from HQ,2023-02-01 14:20:00,NaT


In [87]:
outofstock = outofstock.dropna(subset='Datetimeout')

In [88]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=outofstock.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [89]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=outofstock.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [90]:
outofstock["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60
outofstock =outofstock.replace("", np.nan)

In [91]:
outofstock['delayed dept']=outofstock['delay'].apply(lambda x: 'HQ Item Out of Stock(Designer)' )
des_outofstock=outofstock
des_outofstock.head(5)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
913,approvals1,2023-03-15,13:11:00,Confirmed by Approvals Team,990061,231001439,Frame from HQ and Lens from HQ,2023-03-15 13:11:00,2023-03-15 13:17:00,6.0,HQ Item Out of Stock(Designer)
914,approvals1,2023-03-15,13:11:00,Confirmed by Approvals Team,990061,231001439,Frame from HQ and Lens from HQ,2023-03-15 13:11:00,2023-03-15 13:17:00,6.0,HQ Item Out of Stock(Designer)
1739,approvals2,2023-02-06,10:22:00,Confirmed by Approvals Team,942909,233400699,Overseas Lens with HQ Frame,2023-02-06 10:22:00,2023-02-07 16:00:00,878.0,HQ Item Out of Stock(Designer)
1740,approvals2,2023-02-06,10:22:00,Confirmed by Approvals Team,942909,233400699,Overseas Lens with HQ Frame,2023-02-06 10:22:00,2023-02-07 16:00:00,878.0,HQ Item Out of Stock(Designer)
2346,approvals2,2023-02-09,12:11:00,Confirmed by Approvals Team,953019,234300880,Frame from HQ and Lens from HQ,2023-02-09 12:11:00,2023-02-09 12:31:00,20.0,HQ Item Out of Stock(Designer)


## All Designer

In [92]:
alldesigner=pd.concat([DesignerStore_orderprinted_lenstore,designer_orderprinted_control,designerdamage_reissued,des_outofstock])

In [93]:
alldesigner['delayed dept'].unique()

array(['Did not delay at Designer', 'Delayed at Designer',
       'Designer delayed to reissue Frame',
       'HQ Item Out of Stock(Designer)'], dtype=object)

# Lens store-alllens

## LENSTORE(Branch Frame/Frame sent to lens store to controlroom)-lens_tocontrolroom

In [94]:
Senttolens1 = status[status['Status']== 'Frame Sent to Lens Store']
Senttolens2 = status[status['Status']== 'Branch Frame Sent to Lens Store']
Senttolens3 = status[status['Status']== 'PL Sent to Lens Store']
Senttolens4 = status[status['Status']== 'PF Sent to Lens Store']
Senttolens5 = status[status['Status']== 'PF to Follow Sent to Lens Store']

Senttolens=pd.concat([Senttolens1,Senttolens2,Senttolens3,Senttolens4,Senttolens5])

Senttolens['Status'].unique()

array(['Frame Sent to Lens Store', 'Branch Frame Sent to Lens Store',
       'PL Sent to Lens Store', 'PF Sent to Lens Store',
       'PF to Follow Sent to Lens Store'], dtype=object)

In [95]:
Senttocontrol1 = status[status['Status']== 'ReIssued Lens for Order']
Senttocontrol2 = status[status['Status']== 'Lenses coming from Branch Lab']
Senttocontrol3 = status[status['Status']== 'Blanks Sent to Control Room']
Senttocontrol4 = status[status['Status']== 'Sent to Control Room']


Senttocontrol=pd.concat([Senttocontrol1,Senttocontrol2,Senttocontrol3,Senttocontrol4])

Senttocontrol['Status'].unique()

array(['ReIssued Lens for Order', 'Lenses coming from Branch Lab',
       'Blanks Sent to Control Room', 'Sent to Control Room'],
      dtype=object)

In [96]:
Senttocontrol.rename(columns={'Datetime':'Datetimeout','Status':'Status2'}, inplace=True)

In [97]:
Senttocontrol = Senttocontrol.drop_duplicates(subset=['DocEntry','Status2'], keep='first')

In [98]:
lens_tocontrol= pd.merge(Senttolens,Senttocontrol[['DocNum','Datetimeout','Status2']], on='DocNum', how='left')
lens_tocontrol.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,Status2
0,designer1,2023-02-01,08:57:00,Frame Sent to Lens Store,941526,23300508,Frame from HQ and Lens from HQ,2023-02-01 08:57:00,2023-02-01 09:11:00,Sent to Control Room
1,designer1,2023-02-01,09:29:00,Frame Sent to Lens Store,939090,23203486,Frame from HQ and Lens from HQ,2023-02-01 09:29:00,2023-02-01 09:38:00,Sent to Control Room
2,designer1,2023-02-01,10:51:00,Frame Sent to Lens Store,943533,23800368,Frame from HQ and Lens from HQ,2023-02-01 10:51:00,2023-02-01 10:55:00,Sent to Control Room


In [99]:
lens_tocontrol=lens_tocontrol.dropna(subset='Datetimeout')

In [100]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=lens_tocontrol.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [101]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=lens_tocontrol.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [102]:
lens_tocontrol["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

lens_tocontrol =lens_tocontrol.replace("", np.nan)


In [103]:
lens_tocontrol['delayed dept']=lens_tocontrol['delay'].apply(lambda x: 'Delayed at Lens store' if x>10 else 'Did not delay at Lens store' )
lens_tocontrol['delayed dept'] = np.where(lens_tocontrol['Status2'] =='Lenses coming from Branch Lab','Lenses coming from Branch Lab',lens_tocontrol['delayed dept'])

lens_tocontrol['delayed dept'].unique()

array(['Delayed at Lens store', 'Did not delay at Lens store',
       'Lenses coming from Branch Lab'], dtype=object)

In [104]:
lens_tocontrolroom=lens_tocontrol
lens_tocontrolroom.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,Status2,delay,delayed dept
0,designer1,2023-02-01,08:57:00,Frame Sent to Lens Store,941526,23300508,Frame from HQ and Lens from HQ,2023-02-01 08:57:00,2023-02-01 09:11:00,Sent to Control Room,11.0,Delayed at Lens store
1,designer1,2023-02-01,09:29:00,Frame Sent to Lens Store,939090,23203486,Frame from HQ and Lens from HQ,2023-02-01 09:29:00,2023-02-01 09:38:00,Sent to Control Room,9.0,Did not delay at Lens store


## LENSTORE(PL &PF -Awaiting feedback)-pf_pl_awaiting

In [105]:
Senttolens = status[status['Status']== 'PF & PL Sent to Lens Store']

Senttolens['Status'].unique()

array(['PF & PL Sent to Lens Store'], dtype=object)

In [106]:
Senttocontrol = status[status['Status']== 'Lens Store Awaiting Branch feedback from Client']

Senttocontrol['Status'].unique()

array(['Lens Store Awaiting Branch feedback from Client'], dtype=object)

In [107]:
Senttocontrol.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15940\2763136328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Senttocontrol.rename(columns={'Datetime':'Datetimeout'}, inplace=True)


In [108]:
Senttocontrol = Senttocontrol.drop_duplicates(subset=['DocEntry','Status'], keep='first')

In [109]:
lens_tocontrol= pd.merge(Senttolens,Senttocontrol[['DocNum','Datetimeout']], on='DocNum', how='left')
lens_tocontrol.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,receiving1,2023-02-01,11:56:00,PF & PL Sent to Lens Store,943523,23601560,Spare Part from HQ,2023-02-01 11:56:00,NaT
1,receiving1,2023-02-01,12:20:00,PF & PL Sent to Lens Store,942992,236800320,Spare Part from HQ,2023-02-01 12:20:00,NaT
2,receiving1,2023-02-01,12:28:00,PF & PL Sent to Lens Store,943454,235700149,PL and Branch Frame,2023-02-01 12:28:00,NaT


In [110]:
lens_tocontrol=lens_tocontrol.dropna(subset='Datetimeout')

In [111]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=lens_tocontrol.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [112]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=lens_tocontrol.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [113]:
lens_tocontrol["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

lens_tocontrol =lens_tocontrol.replace("", np.nan)


In [114]:
lens_tocontrol['delayed dept']=lens_tocontrol['delay'].apply(lambda x: 'Client delayed to give feedback' if x>1200 else 'Did not delay at Lens store' )

lens_tocontrol['delayed dept'].unique()

array(['Did not delay at Lens store', 'Client delayed to give feedback'],
      dtype=object)

In [115]:
lens_toawaiting=lens_tocontrol
lens_toawaiting.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
13,receiving1,2023-02-03,09:39:00,PF & PL Sent to Lens Store,945709,233300301,Spare Part from HQ,2023-02-03 09:39:00,2023-02-03 12:48:00,189.0,Did not delay at Lens store
22,receiving1,2023-02-06,09:45:00,PF & PL Sent to Lens Store,948176,234800475,Spare Part from HQ,2023-02-06 09:45:00,2023-02-07 12:01:00,676.0,Did not delay at Lens store


## All lensstore

In [116]:
alllens=pd.concat([lens_tocontrolroom,lens_toawaiting])

In [117]:
alllens['Status'].unique()

array(['Frame Sent to Lens Store', 'Branch Frame Sent to Lens Store',
       'PL Sent to Lens Store', 'PF Sent to Lens Store',
       'PF to Follow Sent to Lens Store', 'PF & PL Sent to Lens Store'],
      dtype=object)

In [118]:
alllens.to_clipboard()

# Control room-allcontrol

### controlroom to preqc - control_preqc

In [119]:
Senttocontrol = status[status['Status']== 'Sent to Control Room']


Senttocontrol['Status'].unique()

array(['Sent to Control Room'], dtype=object)

In [120]:
Senttopre1 = status[status['Status']== 'Sent to Pre Quality']
Senttopre2 = status[status['Status']== 'Sent to Packaging']
Senttopre3 = status[status['Status']== 'Sent to Surfacing']


Senttopre=pd.concat([Senttopre1,Senttopre2,Senttopre3])

Senttopre['Status'].unique()

array(['Sent to Pre Quality', 'Sent to Packaging', 'Sent to Surfacing'],
      dtype=object)

In [121]:
Senttopre.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [122]:
Senttopre = Senttopre.drop_duplicates(subset=['DocEntry'], keep='first')

In [123]:
cont_pre= pd.merge(Senttocontrol,Senttopre[['DocNum','Datetimeout']], on='DocNum', how='left')
cont_pre.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,designer1,2023-02-01,17:56:00,Sent to Control Room,940978,235400524,Contact lens from HQ,2023-02-01 17:56:00,2023-02-01 17:57:00
1,designer1,2023-02-04,14:34:00,Sent to Control Room,948118,231401002,Contact lens from HQ,2023-02-04 14:34:00,2023-02-04 14:45:00
2,designer1,2023-02-18,11:10:00,Sent to Control Room,963485,23206447,Contact lens from HQ,2023-02-18 11:10:00,2023-02-18 11:16:00


In [124]:
cont_pre=cont_pre.dropna(subset='Datetimeout')

In [125]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(19),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)


def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentWk_hrs=cont_pre.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [126]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=cont_pre.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)

In [127]:
cont_pre["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

cont_pre =cont_pre.replace("", np.nan)


In [128]:
cont_pre['delayed dept']=cont_pre['delay'].apply(lambda x: 'Delayed at Control room' if x>10 else 'Did not delay at Control room' )

cont_pre['delayed dept'].unique()

array(['Did not delay at Control room', 'Delayed at Control room'],
      dtype=object)

In [129]:
control_preqc=cont_pre
control_preqc.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,designer1,2023-02-01,17:56:00,Sent to Control Room,940978,235400524,Contact lens from HQ,2023-02-01 17:56:00,2023-02-01 17:57:00,1.0,Did not delay at Control room
1,designer1,2023-02-04,14:34:00,Sent to Control Room,948118,231401002,Contact lens from HQ,2023-02-04 14:34:00,2023-02-04 14:45:00,11.0,Delayed at Control room


### All Control

In [130]:
allcontrol = control_preqc

# Workshop

### Damaged orders - control_damage

In [131]:
control_preqc.to_clipboard()

In [132]:
damtocontrol1 = status[status['Status']== 'Damage Sent to Control Rooom']
damtocontrol2 = status[status['Status']== 'Surfacing Damage/Reject Sent to Control Room']
damtocontrol3 = status[status['Status']== 'Rejected Order Sent To Control Room']


damtocontrol=pd.concat([damtocontrol1,damtocontrol2,damtocontrol3])
damtocontrol['Status'].unique()

array(['Damage Sent to Control Rooom',
       'Surfacing Damage/Reject Sent to Control Room',
       'Rejected Order Sent To Control Room'], dtype=object)

In [133]:
work_store = damtocontrol

In [134]:
work_store['delayed dept']='Rejected/Damaged order'


In [135]:
work_store['delayed dept'].unique()

array(['Rejected/Damaged order'], dtype=object)

## Glazing

In [136]:
damtostore1 = status[status['Status']== 'ReIssued Lens for Order']
damtostore2 = status[status['Status']== 'Rejected Order Sent To Control Room']
damtostore3 = status[status['Status']== 'ReIssued Frame for Order']


damtostore=pd.concat([damtostore1,damtostore2,damtostore3])

damtostore['Status'].unique()

array(['ReIssued Lens for Order', 'Rejected Order Sent To Control Room',
       'ReIssued Frame for Order'], dtype=object)

In [137]:
final1 = status[status['Status']== 'Sent to Packaging']
final2 = status[status['Status']== 'Rejected Order Sent To Control Room']
final3 = status[status['Status']== 'Damage Sent to Control Rooom']


final=pd.concat([final1,final2,final3])

final['Status'].unique()

array(['Sent to Packaging', 'Rejected Order Sent To Control Room',
       'Damage Sent to Control Rooom'], dtype=object)

In [138]:
final.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [139]:
final = final.drop_duplicates(subset=['DocEntry'], keep='first')
damtostore= damtostore.drop_duplicates(subset=['DocEntry'], keep='first')

In [140]:
pre_final= pd.merge(damtostore,final[['DocNum','Datetimeout']], on='DocNum', how='left')
pre_final.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,lens1,2023-02-01,10:02:00,ReIssued Lens for Order,942757,23203935,Order with PF from Branch and Lens from HQ,2023-02-01 10:02:00,2023-02-01 10:59:00
1,lens1,2023-02-01,13:55:00,ReIssued Lens for Order,942948,232800807,Frame from HQ and Lens from HQ,2023-02-01 13:55:00,2023-02-01 14:25:00
2,lens1,2023-02-01,14:40:00,ReIssued Lens for Order,944036,23601582,Frame from HQ and Lens from HQ,2023-02-01 14:40:00,2023-02-01 15:58:00


In [141]:
pre_final=pre_final.dropna(subset='Datetimeout')

In [142]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(19),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)


def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentWk_hrs=pre_final.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [143]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=pre_final.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)

In [144]:
pre_final["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

pre_final =pre_final.replace("", np.nan)


In [145]:
pre_final['delayed dept']=pre_final['delay'].apply(lambda x: 'Delayed at workshop' if x>60 else 'Did not delay at Workshop' )

pre_final['delayed dept'].unique()

array(['Did not delay at Workshop', 'Delayed at workshop'], dtype=object)

In [146]:
pre_final.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,lens1,2023-02-01,10:02:00,ReIssued Lens for Order,942757,23203935,Order with PF from Branch and Lens from HQ,2023-02-01 10:02:00,2023-02-01 10:59:00,57.0,Did not delay at Workshop
1,lens1,2023-02-01,13:55:00,ReIssued Lens for Order,942948,232800807,Frame from HQ and Lens from HQ,2023-02-01 13:55:00,2023-02-01 14:25:00,30.0,Did not delay at Workshop


# Surfacing

In [147]:
con_sur = status[status['Status']== 'Sent to Surfacing']

con_sur['Status'].unique()

array(['Sent to Surfacing'], dtype=object)

In [148]:
sur_pre = status[status['Status']== 'Sent to Pre Quality']

sur_pre['Status'].unique()

array(['Sent to Pre Quality'], dtype=object)

In [149]:
sur_pre.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15940\348650746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sur_pre.rename(columns={'Datetime':'Datetimeout'}, inplace=True)


In [150]:
con_sur = con_sur.drop_duplicates(subset=['DocEntry'], keep='first')
sur_pre= sur_pre.drop_duplicates(subset=['DocEntry'], keep='first')

In [151]:
con_pre= pd.merge(con_sur,sur_pre[['DocNum','Datetimeout']], on='DocNum', how='left')
con_pre.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,control1,2023-02-01,12:36:00,Sent to Surfacing,943756,231900593,Surfacing Lens with HQ Frame,2023-02-01 12:36:00,2023-02-01 13:41:00
1,control1,2023-02-01,15:49:00,Sent to Surfacing,942999,23700290,Surfacing Lens with HQ Frame,2023-02-01 15:49:00,2023-02-01 17:18:00
2,control1,2023-02-01,17:54:00,Sent to Surfacing,933131,232800585,Surfacing Lens with PF,2023-02-01 17:54:00,2023-02-02 10:46:00


In [152]:
con_pre=con_pre.dropna(subset='Datetimeout')

In [153]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(19),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)


def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentWk_hrs=con_pre.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [154]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=con_pre.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)

In [155]:
con_pre["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

con_pre =con_pre.replace("", np.nan)


In [156]:
con_pre['delayed dept']=con_pre['delay'].apply(lambda x: 'Delayed at Surfacing' if x>40 else 'Did not delay at Surfacing' )

con_pre['delayed dept'].unique()

array(['Delayed at Surfacing', 'Did not delay at Surfacing'], dtype=object)

In [157]:
con_pre.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,control1,2023-02-01,12:36:00,Sent to Surfacing,943756,231900593,Surfacing Lens with HQ Frame,2023-02-01 12:36:00,2023-02-01 13:41:00,65.0,Delayed at Surfacing
1,control1,2023-02-01,15:49:00,Sent to Surfacing,942999,23700290,Surfacing Lens with HQ Frame,2023-02-01 15:49:00,2023-02-01 17:18:00,89.0,Delayed at Surfacing


## All workshop - allwork

In [158]:
allwork = pd.concat([con_pre,pre_final,work_store])

## Packaging - final_branch

In [159]:
final_pac1 = status[status['Status']== 'Sent to Packaging']
final_pac2 = status[status['Status']== 'Damaged PF/PL Sent to Packaging']
final_pac3 = status[status['Status']== 'Blanks Sent to Packaging']

final_pac = pd.concat([final_pac1,final_pac2,final_pac3])

final_pac['Status'].unique()

array(['Sent to Packaging', 'Damaged PF/PL Sent to Packaging',
       'Blanks Sent to Packaging'], dtype=object)

In [160]:
pack_bra1 = status[status['Status']== 'Sent to Branch']
pack_bra2 = status[status['Status']== 'Branch Glazing Sent to Branch']
pack_bra3 = status[status['Status']== 'Damaged PF/PL Sent to Branch']

pack_bra = pd.concat([pack_bra1,pack_bra2,pack_bra3])

pack_bra['Status'].unique()

array(['Sent to Branch', 'Branch Glazing Sent to Branch',
       'Damaged PF/PL Sent to Branch'], dtype=object)

In [161]:
pack_bra.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [162]:
final_pac = final_pac.drop_duplicates(subset=['DocEntry'], keep='first')
pack_bra= pack_bra.drop_duplicates(subset=['DocEntry'], keep='first')

In [163]:
final_branch= pd.merge(final_pac,pack_bra[['DocNum','Datetimeout']], on='DocNum', how='left')
final_branch.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,control1,2023-02-01,12:05:00,Sent to Packaging,943666,231500485,Contact lens from HQ,2023-02-01 12:05:00,2023-02-01 13:54:00
1,control1,2023-02-01,13:49:00,Sent to Packaging,943481,233100350,Contact lens from HQ,2023-02-01 13:49:00,2023-02-01 13:54:00
2,control1,2023-02-01,13:49:00,Sent to Packaging,943861,231300735,Contact lens from HQ,2023-02-01 13:49:00,2023-02-01 13:54:00


In [164]:
final_branch=final_branch.dropna(subset='Datetimeout')

In [165]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(19),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)


def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentWk_hrs=final_branch.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [166]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=final_branch.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)

In [167]:
final_branch["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

final_branch =final_branch.replace("", np.nan)

In [168]:
final_branch['delayed dept']=final_branch['delay'].apply(lambda x: 'Delayed at Packaging' if x>10 else 'Did not delay at Packaging' )

final_branch['delayed dept'].unique()

array(['Delayed at Packaging', 'Did not delay at Packaging'], dtype=object)

In [169]:
final_branch.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,control1,2023-02-01,12:05:00,Sent to Packaging,943666,231500485,Contact lens from HQ,2023-02-01 12:05:00,2023-02-01 13:54:00,109.0,Delayed at Packaging
1,control1,2023-02-01,13:49:00,Sent to Packaging,943481,233100350,Contact lens from HQ,2023-02-01 13:49:00,2023-02-01 13:54:00,5.0,Did not delay at Packaging


## All packaging

In [170]:
final_branch = final_branch

# Overseas-alloverseas

In [171]:
Senttoover1 = status[status['Status']== 'Frame Sent to Overseas Desk']
Senttoover2 = status[status['Status']== 'Branch Frame Sent to Overseas Desk']
Senttoover3 = status[status['Status']== 'PF Sent to Overseas Desk']
Senttoover4 = status[status['Status']== 'PF to Follow Sent to Overseas Desk']


Senttoover=pd.concat([Senttoover1,Senttoover2,Senttoover3,Senttoover4])

Senttoover['Status'].unique()

array(['Frame Sent to Overseas Desk',
       'Branch Frame Sent to Overseas Desk', 'PF Sent to Overseas Desk',
       'PF to Follow Sent to Overseas Desk'], dtype=object)

In [172]:
Senttoover.to_clipboard()

In [173]:
oversent = status[status['Status']== 'Generated GRPO']


oversent['Status'].unique()

array(['Generated GRPO'], dtype=object)

In [174]:
oversent.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15940\2301015548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oversent.rename(columns={'Datetime':'Datetimeout'}, inplace=True)


In [175]:
Senttoover = Senttoover.drop_duplicates(subset=['DocEntry','Status'], keep='first')
oversent = oversent.drop_duplicates(subset=['DocEntry','Status'], keep='first')

In [176]:
over_seas= pd.merge(Senttoover,oversent[['DocNum','Datetimeout']], on='DocNum', how='left')
over_seas.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,designer1,2023-02-01,08:57:00,Frame Sent to Overseas Desk,943315,23601553,Overseas Lens with HQ Frame,2023-02-01 08:57:00,2023-02-10 16:59:00
1,designer1,2023-02-01,09:29:00,Frame Sent to Overseas Desk,942437,23203886,Overseas Lens with HQ Frame,2023-02-01 09:29:00,2023-02-10 15:54:00
2,designer1,2023-02-01,09:29:00,Frame Sent to Overseas Desk,941250,23102684,Overseas Lens with HQ Frame,2023-02-01 09:29:00,2023-02-10 16:13:00


In [177]:
over_seas=over_seas.dropna(subset='Datetimeout')

In [178]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=over_seas.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [179]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=over_seas.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [180]:
over_seas["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

over_seas =over_seas.replace("", np.nan)


In [181]:
over_seas['delayed dept']=over_seas['delay'].apply(lambda x: 'Delayed at Customs' if x>6480 else 'Did not delay at Customs' )

over_seas['delayed dept'].unique()

array(['Did not delay at Customs', 'Delayed at Customs'], dtype=object)

## All overseas 

In [182]:
alloverseas = over_seas

# Branches

In [232]:
data = status

In [233]:
data1 = data[data['Status']=='Printed PF Identifier']
data2 = data[data['Status']=='Printed Frame Identifier']
data3 = data[data['Status']=='Printed Lens Identifier']
data4 = data[data['Status']=='Printed Frame and Lens Identifier']
data5 = data[data['Status']=='Printed PF to Follow Identifier']

printed = pd.concat([data1,data2,data3,data4,data5])

printed['Status'].unique()

array(['Printed PF Identifier', 'Printed Frame Identifier',
       'Printed Lens Identifier', 'Printed Frame and Lens Identifier',
       'Printed PF to Follow Identifier'], dtype=object)

In [234]:
printed = printed.rename(columns={'Datetime':'printed_time'})
printed.head()

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,printed_time
137372,ohoc1,2023-02-01,13:17:00,Printed PF Identifier,943895,23204073,Order with PF from Branch and Lens from HQ,2023-02-01 13:17:00
137679,ohoc1,2023-02-02,17:12:00,Printed PF Identifier,945696,23204279,Overseas Lens with PF,2023-02-02 17:12:00
137730,ohoc1,2023-02-03,10:49:00,Printed PF Identifier,946085,23204309,Order with PF from Branch and Lens from HQ,2023-02-03 10:49:00
138151,ohoc1,2023-02-04,15:03:00,Printed PF Identifier,948244,23204577,Order with PF from Branch and Lens from HQ,2023-02-04 15:03:00
138152,ohoc1,2023-02-04,15:03:00,Printed PF Identifier,948254,23204579,Order with PF from Branch and Lens from HQ,2023-02-04 15:03:00


In [235]:
sales1 = data[data['Status']=='Sales Order Created']
sales2 = data[data['Status']=='Confirmed by Approvals Team']

sales = pd.concat([sales1,sales2])

sales['Status'].unique()

array(['Sales Order Created', 'Confirmed by Approvals Team'], dtype=object)

In [236]:
sales = sales.rename(columns={'Datetime':'sales_time'})
sales.head()

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,sales_time
16448,bdman,2023-02-01,09:28:00,Sales Order Created,943250,231201440,PL and HQ Frame,2023-02-01 09:28:00
16451,bdman,2023-02-01,10:04:00,Sales Order Created,940107,23203626,Frame from HQ and Lens from HQ,2023-02-01 10:04:00
16452,bdman,2023-02-01,11:44:00,Sales Order Created,942058,231201416,Frame from HQ and Lens from HQ,2023-02-01 11:44:00
16454,bdman,2023-02-01,13:10:00,Sales Order Created,943294,23204004,Overseas Lens with HQ Frame,2023-02-01 13:10:00
16456,bdman,2023-02-02,08:09:00,Sales Order Created,943763,232600216,OTC,2023-02-02 08:09:00


In [237]:
final = pd.merge(sales,printed[['DocEntry','printed_time']], how='left', on='DocEntry')
final.head(10)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,sales_time,printed_time
0,bdman,2023-02-01,09:28:00,Sales Order Created,943250,231201440,PL and HQ Frame,2023-02-01 09:28:00,NaT
1,bdman,2023-02-01,10:04:00,Sales Order Created,940107,23203626,Frame from HQ and Lens from HQ,2023-02-01 10:04:00,NaT
2,bdman,2023-02-01,11:44:00,Sales Order Created,942058,231201416,Frame from HQ and Lens from HQ,2023-02-01 11:44:00,NaT
3,bdman,2023-02-01,13:10:00,Sales Order Created,943294,23204004,Overseas Lens with HQ Frame,2023-02-01 13:10:00,NaT
4,bdman,2023-02-02,08:09:00,Sales Order Created,943763,232600216,OTC,2023-02-02 08:09:00,NaT
5,bdman,2023-02-02,08:10:00,Sales Order Created,936391,234600222,Frame from HQ and Lens from HQ,2023-02-02 08:10:00,NaT
6,bdman,2023-02-02,08:12:00,Sales Order Created,943124,232400265,Frame from HQ and Lens from HQ,2023-02-02 08:12:00,NaT
7,bdman,2023-02-02,08:12:00,Sales Order Created,943751,236400324,Overseas Lens with HQ Frame,2023-02-02 08:12:00,NaT
8,bdman,2023-02-02,08:13:00,Sales Order Created,943828,232600218,OTC,2023-02-02 08:13:00,NaT
9,bdman,2023-02-02,08:14:00,Sales Order Created,944474,231100745,OTC,2023-02-02 08:14:00,NaT


In [238]:
final = final.dropna(subset='printed_time')
final.head(4)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,sales_time,printed_time
13,bdman,2023-02-02,09:11:00,Sales Order Created,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00
14,bdman,2023-02-02,09:11:00,Sales Order Created,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00
67,bdman,2023-02-08,09:49:00,Sales Order Created,946622,23103098,Overseas Lens with Branch Frame,2023-02-08 09:49:00,2023-02-08 09:54:00
68,bdman,2023-02-08,09:49:00,Sales Order Created,946622,23103098,Overseas Lens with Branch Frame,2023-02-08 09:49:00,2023-02-08 09:54:00


In [239]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=final.apply(lambda row: BusHrs(row['sales_time'], row['printed_time']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [240]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=final.apply(lambda row: BusHrs(row['sales_time'], row['printed_time']), axis=1)


In [241]:
final['Time difference'] =(RejectedSentWk_hrs+RejectedSentSat_hrs)*60
final.head(5)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,sales_time,printed_time,Time difference
13,bdman,2023-02-02,09:11:00,Sales Order Created,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00,4201.0
14,bdman,2023-02-02,09:11:00,Sales Order Created,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00,4201.0
67,bdman,2023-02-08,09:49:00,Sales Order Created,946622,23103098,Overseas Lens with Branch Frame,2023-02-08 09:49:00,2023-02-08 09:54:00,5.0
68,bdman,2023-02-08,09:49:00,Sales Order Created,946622,23103098,Overseas Lens with Branch Frame,2023-02-08 09:49:00,2023-02-08 09:54:00,5.0
111,bdman,2023-02-15,16:29:00,Sales Order Created,960511,23206067,Overseas Lens with Branch Frame,2023-02-15 16:29:00,2023-02-15 16:31:00,2.0


In [242]:
final['delayed dept'] = final['Time difference'].apply(lambda x: 'Delayed to print Identifier' if x > 5 else 'Did not delay to print Identifier')
final.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,sales_time,printed_time,Time difference,delayed dept
13,bdman,2023-02-02,09:11:00,Sales Order Created,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00,4201.0,Delayed to print Identifier
14,bdman,2023-02-02,09:11:00,Sales Order Created,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00,4201.0,Delayed to print Identifier
67,bdman,2023-02-08,09:49:00,Sales Order Created,946622,23103098,Overseas Lens with Branch Frame,2023-02-08 09:49:00,2023-02-08 09:54:00,5.0,Did not delay to print Identifier


In [243]:
final['Status'] = final['Status'].apply(lambda x: 'Cash order' if x=='Sales Order Created' else 'Insurance order')

In [244]:
final = final.drop(columns={'Date','Time'})
final = final.rename(columns={'Status':'Payment'})
final.head(5)

,Created User,Payment,DocEntry,DocNum,OrderCriteria Status,sales_time,printed_time,Time difference,delayed dept
13,bdman,Cash order,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00,4201.0,Delayed to print Identifier
14,bdman,Cash order,944259,23102889,Overseas Lens with PF to Follow,2023-02-02 09:11:00,2023-02-11 09:12:00,4201.0,Delayed to print Identifier
67,bdman,Cash order,946622,23103098,Overseas Lens with Branch Frame,2023-02-08 09:49:00,2023-02-08 09:54:00,5.0,Did not delay to print Identifier
68,bdman,Cash order,946622,23103098,Overseas Lens with Branch Frame,2023-02-08 09:49:00,2023-02-08 09:54:00,5.0,Did not delay to print Identifier
111,bdman,Cash order,960511,23206067,Overseas Lens with Branch Frame,2023-02-15 16:29:00,2023-02-15 16:31:00,2.0,Did not delay to print Identifier


In [282]:
final =final.rename(columns={'printed_time':'Datetime','sales_time':'Datetimeout'})

In [283]:
final = final[['Created User','DocEntry','Datetime','Datetimeout','Time difference','delayed dept']]
final.head(2)

,Created User,DocEntry,Datetime,Datetimeout,Time difference,delayed dept
13,bdman,944259,2023-02-11 09:12:00,2023-02-02 09:11:00,4201.0,Delayed to print Identifier
14,bdman,944259,2023-02-11 09:12:00,2023-02-02 09:11:00,4201.0,Delayed to print Identifier


## Printed Identifier - sent to hq

In [246]:
data1 = data[data['Status']=='Printed PF Identifier']
data2 = data[data['Status']=='Printed Frame Identifier']
data3 = data[data['Status']=='Printed Lens Identifier']
data4 = data[data['Status']=='Printed Frame and Lens Identifier']
data5 = data[data['Status']=='Printed PF to Follow Identifier']

printed = pd.concat([data1,data2,data3,data4,data5])

printed['Status'].unique()

array(['Printed PF Identifier', 'Printed Frame Identifier',
       'Printed Lens Identifier', 'Printed Frame and Lens Identifier',
       'Printed PF to Follow Identifier'], dtype=object)

In [247]:
data1 = data[data['Status']=='PF & PL Sent to HQ']
data2 = data[data['Status']=='PF Sent to HQ']
data3 = data[data['Status']=='PL Sent to HQ']
data4 = data[data['Status']=='Branch Frame Sent to HQ']
data5 = data[data['Status']=='PF to Follow Sent to HQ']

senthq = pd.concat([data1,data2,data3,data4,data5])

senthq['Status'].unique()

array(['PF & PL Sent to HQ', 'PF Sent to HQ', 'PL Sent to HQ',
       'Branch Frame Sent to HQ', 'PF to Follow Sent to HQ'], dtype=object)

In [248]:
## printed
printed['Date'] = pd.to_datetime(printed['Date'],dayfirst=True)
printed['datetime'] = printed['Date'].astype(str) +' '+printed['Time'].astype(str)
printed['datetime'] = pd.to_datetime(printed['datetime'], format = '%Y/%m/%d %H:%M:%S')
printed = printed.rename(columns={'datetime':'printedtime'})
printed.head(4)


## Sent to hq
senthq['Date'] = pd.to_datetime(senthq['Date'],dayfirst=True)
senthq['datetime'] = senthq['Date'].astype(str) +' '+senthq['Time'].astype(str)
senthq['datetime'] = pd.to_datetime(senthq['datetime'], format = '%Y/%m/%d %H:%M:%S')
senthq = senthq.rename(columns={'datetime':'senthqtime'})
senthq.head(4)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,senthqtime
137377,ohoc1,2023-02-01,13:25:00,PF & PL Sent to HQ,943917,23204075,Spare Part from HQ,2023-02-01 13:25:00,2023-02-01 13:25:00
138058,ohoc1,2023-02-04,12:45:00,PF & PL Sent to HQ,947790,23204523,Spare Part from HQ,2023-02-04 12:45:00,2023-02-04 12:45:00
138432,ohoc1,2023-02-06,15:54:00,PF & PL Sent to HQ,949840,23204745,Spare Part from HQ,2023-02-06 15:54:00,2023-02-06 15:54:00
138482,ohoc1,2023-02-06,17:17:00,PF & PL Sent to HQ,950063,23204776,Spare Part from HQ,2023-02-06 17:17:00,2023-02-06 17:17:00


In [249]:
sent = pd.merge(printed,senthq[['DocEntry','senthqtime']], on='DocEntry', how='left')
sent.head(5)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,printedtime,senthqtime
0,ohoc1,2023-02-01,13:17:00,Printed PF Identifier,943895,23204073,Order with PF from Branch and Lens from HQ,2023-02-01 13:17:00,2023-02-01 13:17:00,2023-02-01 13:18:00
1,ohoc1,2023-02-01,13:17:00,Printed PF Identifier,943895,23204073,Order with PF from Branch and Lens from HQ,2023-02-01 13:17:00,2023-02-01 13:17:00,2023-02-01 13:18:00
2,ohoc1,2023-02-02,17:12:00,Printed PF Identifier,945696,23204279,Overseas Lens with PF,2023-02-02 17:12:00,2023-02-02 17:12:00,2023-02-02 17:12:00
3,ohoc1,2023-02-02,17:12:00,Printed PF Identifier,945696,23204279,Overseas Lens with PF,2023-02-02 17:12:00,2023-02-02 17:12:00,2023-02-02 17:12:00
4,ohoc1,2023-02-03,10:49:00,Printed PF Identifier,946085,23204309,Order with PF from Branch and Lens from HQ,2023-02-03 10:49:00,2023-02-03 10:49:00,2023-02-03 10:50:00


In [250]:
##
sent = sent.dropna(subset='senthqtime')
sent.head(5)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,printedtime,senthqtime
0,ohoc1,2023-02-01,13:17:00,Printed PF Identifier,943895,23204073,Order with PF from Branch and Lens from HQ,2023-02-01 13:17:00,2023-02-01 13:17:00,2023-02-01 13:18:00
1,ohoc1,2023-02-01,13:17:00,Printed PF Identifier,943895,23204073,Order with PF from Branch and Lens from HQ,2023-02-01 13:17:00,2023-02-01 13:17:00,2023-02-01 13:18:00
2,ohoc1,2023-02-02,17:12:00,Printed PF Identifier,945696,23204279,Overseas Lens with PF,2023-02-02 17:12:00,2023-02-02 17:12:00,2023-02-02 17:12:00
3,ohoc1,2023-02-02,17:12:00,Printed PF Identifier,945696,23204279,Overseas Lens with PF,2023-02-02 17:12:00,2023-02-02 17:12:00,2023-02-02 17:12:00
4,ohoc1,2023-02-03,10:49:00,Printed PF Identifier,946085,23204309,Order with PF from Branch and Lens from HQ,2023-02-03 10:49:00,2023-02-03 10:49:00,2023-02-03 10:50:00


In [251]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(10),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=sent.apply(lambda row: BusHrs(row['printedtime'], row['senthqtime']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [252]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(10),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=sent.apply(lambda row: BusHrs(row['printedtime'], row['senthqtime']), axis=1)


In [253]:
sent['Time difference'] =(RejectedSentWk_hrs+RejectedSentSat_hrs)*60
sent.head(5)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,printedtime,senthqtime,Time difference
0,ohoc1,2023-02-01,13:17:00,Printed PF Identifier,943895,23204073,Order with PF from Branch and Lens from HQ,2023-02-01 13:17:00,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0
1,ohoc1,2023-02-01,13:17:00,Printed PF Identifier,943895,23204073,Order with PF from Branch and Lens from HQ,2023-02-01 13:17:00,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0
2,ohoc1,2023-02-02,17:12:00,Printed PF Identifier,945696,23204279,Overseas Lens with PF,2023-02-02 17:12:00,2023-02-02 17:12:00,2023-02-02 17:12:00,0.0
3,ohoc1,2023-02-02,17:12:00,Printed PF Identifier,945696,23204279,Overseas Lens with PF,2023-02-02 17:12:00,2023-02-02 17:12:00,2023-02-02 17:12:00,0.0
4,ohoc1,2023-02-03,10:49:00,Printed PF Identifier,946085,23204309,Order with PF from Branch and Lens from HQ,2023-02-03 10:49:00,2023-02-03 10:49:00,2023-02-03 10:50:00,1.0


In [284]:
sent =sent.rename(columns={'printedtime':'Datetime','senthqtime':'Datetimeout'})

In [288]:
sent = sent[['Created User','DocEntry','Datetime','Datetimeout','Time difference','delayed dept']]
sent.head(5)

,Created User,DocEntry,Datetime,Datetimeout,Time difference,delayed dept
0,ohoc1,943895,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0,Not delay to send to hq
1,ohoc1,943895,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0,Not delay to send to hq
2,ohoc1,945696,2023-02-02 17:12:00,2023-02-02 17:12:00,0.0,Not delay to send to hq
3,ohoc1,945696,2023-02-02 17:12:00,2023-02-02 17:12:00,0.0,Not delay to send to hq
4,ohoc1,946085,2023-02-03 10:49:00,2023-02-03 10:50:00,1.0,Not delay to send to hq


In [289]:
sent['delayed dept'] = sent['Time difference'].apply(lambda x: 'Delayed to send to hq' if x>5 else 'Not delay to send to hq')
sent.head(5)

,Created User,DocEntry,Datetime,Datetimeout,Time difference,delayed dept
0,ohoc1,943895,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0,Not delay to send to hq
1,ohoc1,943895,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0,Not delay to send to hq
2,ohoc1,945696,2023-02-02 17:12:00,2023-02-02 17:12:00,0.0,Not delay to send to hq
3,ohoc1,945696,2023-02-02 17:12:00,2023-02-02 17:12:00,0.0,Not delay to send to hq
4,ohoc1,946085,2023-02-03 10:49:00,2023-02-03 10:50:00,1.0,Not delay to send to hq


## Customer payment

### Insurance

In [256]:
draft = status[status['Status']== 'Customer Coming to do SMART']

draft['Status'].unique()

array([], dtype=object)

In [257]:
confirmed = status[status['Status']== 'Confirmed by Approvals Team']

confirmed['Status'].unique()

array(['Confirmed by Approvals Team'], dtype=object)

In [258]:
confirmed.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15940\405843896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confirmed.rename(columns={'Datetime':'Datetimeout'}, inplace=True)


In [259]:
draft = draft.drop_duplicates(subset=['DocEntry','Status'], keep='first')
confirmed = confirmed.drop_duplicates(subset=['DocEntry','Status'], keep='first')

In [260]:
draft_confirmed= pd.merge(draft,confirmed[['DocNum','Datetimeout']], on='DocNum', how='left')
draft_confirmed.head(3)

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout


In [261]:
draft_confirmed.shape

(0, 9)

In [262]:
draft_confirmed = draft_confirmed.drop_duplicates(subset='DocEntry')

In [263]:
draft_confirmed=draft_confirmed.dropna(subset='Datetimeout')

In [264]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=draft_confirmed.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [265]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=draft_confirmed.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [266]:
draft_confirmed["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

draft_confirmed =draft_confirmed.replace("", np.nan)

In [267]:
draft_confirmed['delayed dept']=draft_confirmed['delay'].apply(lambda x: 'Delayed at Insurance' if x>10 else 'Did not delay at Insurance' )

draft_confirmed['delayed dept'].unique()

array([], dtype=float64)

### Cah orders

In [268]:
draft = status[status['Status']== 'Draft Order Created']

draft['Status'].unique()

array(['Draft Order Created'], dtype=object)

In [269]:
sales_created = status[status['Status']== 'Sales Order Created']

sales_created['Status'].unique()

array(['Sales Order Created'], dtype=object)

In [270]:
sales_created.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15940\2825327139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_created.rename(columns={'Datetime':'Datetimeout'}, inplace=True)


In [271]:
draft = draft.drop_duplicates(subset=['DocEntry','Status'], keep='first')
sales_created = sales_created.drop_duplicates(subset=['DocEntry','Status'], keep='first')

In [272]:
draft_sales= pd.merge(draft,sales_created[['DocNum','Datetimeout']], on='DocNum', how='left')
draft_sales.head(2)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,bdman,2023-02-03,17:30:00,Draft Order Created,947124,231201536,NaN,2023-02-03 17:30:00,NaT
1,csman,2023-03-06,13:21:00,Draft Order Created,981137,23603264,NaN,2023-03-06 13:21:00,NaT


In [273]:
draft_sales=draft_sales.dropna(subset='Datetimeout')

In [274]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=draft_sales.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [275]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=draft_sales.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [276]:
draft_sales["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

draft_sales =final_branch.replace("", np.nan)

In [277]:
draft_sales['delayed dept']=draft_sales['delay'].apply(lambda x: 'Customer delayed to make payment' if x>10 else 'Did not delay at Branch' )

draft_sales['delayed dept'].unique()

array(['Customer delayed to make payment', 'Did not delay at Branch'],
      dtype=object)

## All branch

In [293]:
allbranch1 = pd.concat([draft_confirmed,draft_sales])
allbranch1 = allbranch1.rename(columns={'delay':'Time difference',})
allbranch1.head(2)

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout,Time difference,delayed dept
0,control1,2023-02-01,12:05:00,Sent to Packaging,943666,Contact lens from HQ,2023-02-01 12:05:00,231500485,2023-02-01 13:54:00,109.0,Customer delayed to make payment
1,control1,2023-02-01,13:49:00,Sent to Packaging,943481,Contact lens from HQ,2023-02-01 13:49:00,233100350,2023-02-01 13:54:00,5.0,Did not delay at Branch


In [294]:
allbranch = pd.concat([sent,final,allbranch1])
allbranch = allbranch[['Created User','DocEntry','Datetime','Datetimeout','Time difference','delayed dept']]
allbranch.head(3)

,Created User,DocEntry,Datetime,Datetimeout,Time difference,delayed dept
0,ohoc1,943895,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0,Not delay to send to hq
1,ohoc1,943895,2023-02-01 13:17:00,2023-02-01 13:18:00,1.0,Not delay to send to hq
2,ohoc1,945696,2023-02-02 17:12:00,2023-02-02 17:12:00,0.0,Not delay to send to hq


In [295]:
import xlsxwriter

with pd.ExcelWriter(r"C:\Users\user\Documents\Alex\STB Report\note book\Mitigation\RESULTS.xlsx", engine='xlsxwriter') as writer:    
    allbranch.to_excel(writer, sheet_name='Branch delays', index=False)
    allmain.to_excel(writer, sheet_name='Mainstore', index = False)
    alldesigner.to_excel(writer, sheet_name='Designer Store', index = False)
    alllens.to_excel(writer, sheet_name='Lens store', index = False)
    allcontrol.to_excel(writer, sheet_name='Control room', index = False)
    allwork.to_excel(writer, sheet_name='Workshop', index = False)
    final_branch.to_excel(writer, sheet_name='Packaging', index = False)
    alloverseas.to_excel(writer, sheet_name='Overseas', index = False)    
    
    